In [48]:
#HF Example

import torch
from transformers import pipeline

pipe = pipeline(
    task="image-classification",
    model="facebook/dinov2-small-imagenet1k-1-layer",
    dtype=torch.float16,
    device="cuda"
)

pipe("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/pipeline-cat-chonk.jpeg")

/home/adam/anaconda3/envs/CLIP/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Device set to use cuda


[{'label': 'tiger, Panthera tigris', 'score': 0.24335609376430511},
 {'label': 'tiger cat', 'score': 0.24146227538585663},
 {'label': 'lynx, catamount', 'score': 0.16084855794906616},
 {'label': 'marmot', 'score': 0.043973542749881744},
 {'label': 'tabby, tabby cat', 'score': 0.03293466567993164}]

In [ ]:


from transformers import AutoImageProcessor, AutoModel
from PIL import Image
import requests

url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)

processor = AutoImageProcessor.from_pretrained('facebook/dinov2-base')
model = AutoModel.from_pretrained('facebook/dinov2-base')

inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)
last_hidden_states = outputs.last_hidden_state
cls_embedding = last_hidden_states[:, 0, :]



In [50]:
print(cls_embedding.size())

torch.Size([1, 768])


In [40]:
file=hdf5.open_HDF5("downloaded_embeddings.h5")
vectors=file["vectors"]
vectors=vectors[:] #load into a np array
labels=file["coordinates"]
labels=labels[:]
print(vectors)
print(labels)
file.close()

[[-3.8935735   0.5611535   0.7221834  ... -1.6112721  -1.7305301
  -1.24625   ]
 [ 0.959288    1.5554346  -0.45038044 ...  0.38674653 -1.9512769
   0.13411409]
 [ 0.90353054  4.137877    1.1482705  ... -3.668721    0.68040836
   1.5894419 ]
 ...
 [ 1.7816428  -0.3815659   0.29235205 ... -0.32355672  0.05422599
   0.9596687 ]
 [-1.0558149   0.5838056  -1.5675911  ... -1.1514442   0.01467256
  -1.4091729 ]
 [ 0.10558449  1.7960936  -1.2238214  ... -0.98181576  2.1069787
  -1.3743852 ]]
[[48.284294  4.02083 ]
 [37.122986 -3.568118]
 [52.24937   8.912947]
 ...
 [44.60789  11.12855 ]
 [50.534054 -3.564082]
 [50.379196  4.45012 ]]


In [ ]:
import h5py
import hdf5
#Size of hp5
print("agah")
path="/home/adam/source/CLIP/full_dataset_embeddings.h5"
hdf5.get_size(path)




coordinates: shape=(3167055, 2), dtype=float32
vectors: shape=(3167055, 768), dtype=float32


In [ ]:
#Test the clip model


import utils
import importlib
import torch.nn as nn
import torch
importlib.reload(utils)
from tqdm import tqdm

#for having the right model; print the model if unsurea about the layers
dim_fourier_encoding=64 #multiple of 4!!
dim_hidden=256
dim_emb=128 #this one is actually shared with img embeddings
device="cuda"
data_path="/home/adam/source/CLIP/full_dataset_embeddings.h5"
#dim image layer size: 
#As of now: linear from 768 to dim_emb. We could also have MLP if non-linearity needed

image_encoder=nn.Linear(768,dim_emb).to(device)
pos_encoder=utils.Fourier_MLP(original_dim=2, fourier_dim=dim_fourier_encoding, hidden_dim=dim_hidden, output_dim=dim_emb).to(device)

model= utils.DoubleNetwork(image_encoder,pos_encoder).to(device)
model.load_state_dict(torch.load("/home/adam/source/CLIP/save_14_to_17-10-25/model.pt", weights_only=True))
model.eval()

nbr_iter=500000
mean_sim, mean_asim, std_sim, std_asim = utils.test_similarity(data_path,model, nbr_samples=2,device="cuda",nbr_iter=nbr_iter,plot=True)
print("Mean:", mean_sim, mean_asim)
print("Std:", std_sim, std_asim)
#for the whole test  set:
#Mean: tensor(0.1810, device='cuda:0', grad_fn=<DivBackward0>) tensor(0.0827, device='cuda:0', grad_fn=<DivBackward0>)
#Std: tensor(0.0645, device='cuda:0', grad_fn=<SqrtBackward0>) tensor(0.0726, device='cuda:0', grad_fn=<SqrtBackward0>)